# Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import string
import re


import codecs

import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.graph_objects as go

from sklearn import feature_extraction
from sklearn import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression


import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import warnings
warnings.simplefilter("ignore")

In [5]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [6]:
new_train = train
new_test = test

In [7]:
df_train= new_train
df_test = new_test

## Performing some EDA

In [8]:
df_train.head(10)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...
5,nso,dinyakišišo tše tša go dirwa gabedi ka ngwaga ...
6,tsn,kgetse nngwe le nngwe e e sa faposiwang mo tsh...
7,ven,mbadelo dze dza laelwa dzi do kwama mahatulele...
8,nso,maloko a dikhuduthamaga a ikarabela mongwe le ...
9,tsn,fa le dirisiwa lebone le tshwanetse go bontsha...


In [9]:
df_train.tail(10)

,lang_id,text
32990,eng,government has a long-term programme for the u...
32991,nso,mo kgopelo e dirilwego go ya ka karolo ya go b...
32992,zul,a umqondisi-jikelele azise umuntu okukhulunywa...
32993,nso,molawana o akaretša mesepelo ka moka ya baname...
32994,eng,manuel marin s ill-fated debt sources but very...
32995,tsn,popo ya dipolateforomo tse ke go tlisa boetele...
32996,sot,modise mosadi na o ntse o sa utlwe hore thaban...
32997,eng,closing date for the submission of completed t...
32998,xho,nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999,sot,mafapha a mang le ona a lokela ho etsa ditlale...


In [10]:
df_test.head(20)

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.
5,6,"Ke feela dilense tše hlakilego, tša pono e tee..."
6,7,<fn>(762010101403 AM) 1495 Final Gems Birthing...
7,8,Ntjhafatso ya konteraka ya mosebetsi: Etsa bon...
8,9,u-GEMS uhlinzeka ngezinzuzo zemithi yezifo ezi...
9,10,"So, on occasion, are statistics misused."


In [11]:
df_test.tail(5)

,index,text
5677,5678,You mark your ballot in private.
5678,5679,Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679,5680,"E Ka kopo etsa kgetho ya hao ka hloko, hobane ..."
5680,5681,"TB ke bokudi ba PMB, mme Morero o tla lefella ..."
5681,5682,Vakatjhela iwebhusayidi yethu ku-www.


In [12]:
df_train.shape

(33000, 2)

In [14]:
df_test.shape

(5682, 2)

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [16]:
df_train["lang_id"].value_counts()

eng    3000
ven    3000
nso    3000
tsn    3000
afr    3000
zul    3000
nbl    3000
xho    3000
tso    3000
ssw    3000
sot    3000
Name: lang_id, dtype: int64

# Separating Independent and Dependent features 

## Text Pre-processing 

Removing any noise from the dataset

In [18]:
## Remove urls
print ('Removing URLs...')
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df_train['text'] = df_train['text'].replace(to_replace = pattern_url, value = subs_url, regex = True)

# Make lower case
print ('Lowering case...')
df_train['text'] = df_train['text'].str.lower()

# Remove punctuation
import string
print ('Cleaning punctuation...')
def remove_punctuation_numbers(post):
    punc_numbers = string.punctuation + '0123456789'
    return ''.join([l for l in post if l not in punc_numbers])
df_train['text'] = df_train['text'].apply(remove_punctuation_numbers)

Removing URLs...
Lowering case...
Cleaning punctuation...


In [19]:
X = df_train['text']
y = df_train['lang_id']

In [20]:
vector = TfidfVectorizer(sublinear_tf=True)
vector = TfidfVectorizer(ngram_range=(1,2), min_df=2)
X =vector.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,  random_state = 56)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0125, random_state=48)

# Making prediction using logistics Regression (LR)

In [22]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

lr_prediction = lr.predict(X_test)
lr_accuracy = accuracy_score(y_test,lr_prediction)

print("Training accuracy Score    : ",lr.score(X_train,y_train))
print("Testing accuracy Score : ",lr_accuracy )
print(classification_report(lr_prediction,y_test))

Training accuracy Score    :  0.9994318181818181
Testing accuracy Score :  0.9965151515151515
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       611
         eng       1.00      1.00      1.00       587
         nbl       0.99      0.99      0.99       607
         nso       1.00      1.00      1.00       594
         sot       1.00      1.00      1.00       576
         ssw       1.00      1.00      1.00       643
         tsn       1.00      1.00      1.00       613
         tso       1.00      1.00      1.00       607
         ven       1.00      1.00      1.00       597
         xho       1.00      0.98      0.99       588
         zul       0.98      0.99      0.99       577

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



# Improving the Logistc Model

In [23]:
# Import the scaler module
from sklearn import preprocessing
scaler = preprocessing.MaxAbsScaler()

# Scale data
X_scaled = scaler.fit_transform(X)

In [24]:
# Split into train and test
X1_train, X1_test, y1_train, y1_test = train_test_split(X_scaled, y, test_size=0.2, random_state=10)

In [25]:
lm_f = LogisticRegression()

In [26]:
# No transformations necessary
lm_f.fit(X1_train, y1_train)

LogisticRegression()

Generate predictions from full model

In [28]:
pred_lm_full = lm_f.predict(X1_test)

lm_full_accuracy = accuracy_score(y1_test,pred_lm_full)

print("Training accuracy Score    : ",lm_f.score(X1_train,y1_train))
print("Testing accuracy Score : ",lm_full_accuracy )
print(classification_report(pred_lm_full,y1_test))

Training accuracy Score    :  1.0
Testing accuracy Score :  0.9966666666666667
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       572
         eng       1.00      1.00      1.00       612
         nbl       0.98      1.00      0.99       559
         nso       1.00      1.00      1.00       574
         sot       1.00      1.00      1.00       567
         ssw       1.00      0.99      0.99       648
         tsn       1.00      1.00      1.00       587
         tso       1.00      1.00      1.00       615
         ven       1.00      1.00      1.00       624
         xho       1.00      1.00      1.00       606
         zul       0.99      0.99      0.99       636

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [33]:
y3 = df_train['lang_id']
y3 = y3.iloc[:5682]
test2 = df_test['text']
test2 = test2.iloc[:5682]
X3 = test2
X1_test =vector.transform(X3)

test_pred = lm_f.predict(X1_test)

y3 = test_pred
test['lang_id'] = test_pred

submit= test[['index','lang_id']]
submit.to_csv('Submit.csv', index=False)

In [30]:
submit

,index,lang_id
0,1,ssw
1,2,nbl
2,3,xho
3,4,ssw
4,5,ssw
...,...,...
5677,5678,ssw
5678,5679,nso
5679,5680,sot
5680,5681,sot


The multinomial Naive Bayes classifier

In [31]:
NB = MultinomialNB()

NB.fit(X_train,y_train)

NB_prediction = NB.predict(X_test)

NB_accuracy = accuracy_score(y_test,NB_prediction)

print("training accuracy Score: ", NB.score(X_train,y_train))
print("Testing accuracy Score : ", NB_accuracy )

print(classification_report(y_test,NB_prediction))

training accuracy Score:  0.9995833333333334
Testing accuracy Score :  0.9986363636363637
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       611
         eng       1.00      1.00      1.00       587
         nbl       1.00      0.99      1.00       611
         nso       1.00      1.00      1.00       594
         sot       1.00      1.00      1.00       576
         ssw       1.00      1.00      1.00       641
         tsn       1.00      1.00      1.00       613
         tso       1.00      1.00      1.00       607
         ven       1.00      1.00      1.00       597
         xho       1.00      1.00      1.00       579
         zul       0.99      1.00      1.00       584

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [34]:
test1 = test['text']
X_test =vector.transform(test1)

test_pred = NB.predict(X_test)

test['lang_id'] = test_pred

submit = test[['index','lang_id']]
submit.to_csv('Submission_2.csv', index=False) 

In [ ]:
submit

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr
...,...,...
5677,5678,eng
5678,5679,nso
5679,5680,sot
5680,5681,sot


In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train,y_train)

rf_prediction = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test,rf_prediction)

print("Training accuracy Score    : ",rf.score(X_train,y_train))
print("Testing accuracy Score : ",rf_accuracy )

print(classification_report(rf_prediction,y_test))

In [35]:
test5 = test['text']
X_test5 =vector.transform(test5)

test5_pred = NB.predict(X_test5)

test['lang_id'] = test5_pred

submit = test[['index','lang_id']]
submit.to_csv('Submission_3.csv', index=False) 